In [3]:
# import mendable as md
import dotenv
import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
import requests
import asyncio
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')


# Might need this later
# MENDABLE_API_KEY = os.getenv("MENDABLE_API_KEY_SPO_TOOL")

[nltk_data] Downloading package punkt to /Users/waelivie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/waelivie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Training the SEADOG model

- We need to first get all the answers from telegram chat csv file and separate the questions and answers. I think the solution I can try to do this is to use nltk library to help determine each. I have googled and thought of a few words that can be used to determine if the sentence is a question or answer to a question. 

- Now that we have the words in arrays, we can now use that to determine if the sentence is a question or answer from our telegram chat csv file.

In [4]:
tg_chat_df = pd.read_csv("armada_chat_test.csv")

In [5]:
tg_chat_df

,Unnamed: 0,sender_id,text,date,message_id,reply,reply_to_message_id
0,0,655534778,Found it... clearly didn't search very hard ba...,2023-12-12 22:42:15+00:00,57624,No,NaN
1,1,655534778,Hey yes I gave it a go to fix my SSD issue. Un...,2023-12-12 22:24:50+00:00,57623,Yes,"MessageReplyHeader(reply_to_scheduled=False, f..."
2,2,2076101368,lol I agree,2023-12-12 21:56:40+00:00,57622,No,NaN
3,3,635318013,Just gotta wait past 35 hours,2023-12-12 21:55:37+00:00,57621,No,NaN
4,4,635318013,"But for the most part, it's mostly stable now",2023-12-12 21:55:27+00:00,57620,No,NaN
...,...,...,...,...,...,...,...
1995,1995,6137081769,NaN,2023-07-23 05:36:52+00:00,55587,No,NaN
1996,1996,1474655618,Would also appreciate a critical review for th...,2023-07-21 20:35:43+00:00,55586,No,NaN
1997,1997,1474655618,https://docs.google.com/document/d/1bSNKml6JW-...,2023-07-21 20:35:28+00:00,55585,No,NaN
1998,1998,1474655618,"Question for you guys, I’m working on a new co...",2023-07-21 20:35:17+00:00,55584,No,NaN


In [6]:
# drop the first column
tg_chat_df = tg_chat_df.drop(tg_chat_df.columns[0], axis=1)

In [7]:
tg_chat_df['reply_to_message_id'][1]

'MessageReplyHeader(reply_to_scheduled=False, forum_topic=False, quote=False, reply_to_msg_id=57582, reply_to_peer_id=None, reply_from=None, reply_media=None, reply_to_top_id=49888, quote_text=None, quote_entities=[], quote_offset=None)'

In [13]:
# we are dropping all NaN rows in the text column from the data set
tg_chat_df = tg_chat_df.dropna(subset=['text'])

# Reset the index
tg_chat_df = tg_chat_df.reset_index(drop=True)


In [14]:
tg_chat_df

,sender_id,text,date,message_id,reply,reply_to_message_id
0,655534778,Found it... clearly didn't search very hard ba...,2023-12-12 22:42:15+00:00,57624,No,NaN
1,655534778,Hey yes I gave it a go to fix my SSD issue. Un...,2023-12-12 22:24:50+00:00,57623,Yes,"MessageReplyHeader(reply_to_scheduled=False, f..."
2,2076101368,lol I agree,2023-12-12 21:56:40+00:00,57622,No,NaN
3,635318013,Just gotta wait past 35 hours,2023-12-12 21:55:37+00:00,57621,No,NaN
4,635318013,"But for the most part, it's mostly stable now",2023-12-12 21:55:27+00:00,57620,No,NaN
...,...,...,...,...,...,...
1899,5403145942,"everything is fine, except block propagation d...",2023-07-23 22:19:31+00:00,55589,No,NaN
1900,5403145942,Hi @WCat13 I got my metrics back! You were rig...,2023-07-23 22:16:20+00:00,55588,No,NaN
1901,1474655618,Would also appreciate a critical review for th...,2023-07-21 20:35:43+00:00,55586,No,NaN
1902,1474655618,https://docs.google.com/document/d/1bSNKml6JW-...,2023-07-21 20:35:28+00:00,55585,No,NaN


In [15]:
tg_chat_df['text'][:1000]

0      Found it... clearly didn't search very hard ba...
1      Hey yes I gave it a go to fix my SSD issue. Un...
2                                            lol I agree
3                          Just gotta wait past 35 hours
4          But for the most part, it's mostly stable now
                             ...                        
995    Oh I spoke to whoever runs the Twitter and the...
996    We can also add audio which is sort of rare. M...
997    Any pool here want to use the AA space on http...
998    Down and starting again so could be hours befo...
999    Both relays are down and starting at the momen...
Name: text, Length: 1000, dtype: object

Okay, so how can I rank the questions or find a way to determine the most meaningful questions.... I am currently going to try and just check frequency a question or similar questions are asked.

I am not sure what project to use to help with my project. There are a few things I am thinking of using and I have a certain strategy in mind.


Developing a program to filter and rank valuable questions and answers from a chat group using the Telegram API is a multifaceted task. Your current approach, based on identifying questions and answers through keywords, is a good start. However, there are several other techniques and considerations that can enhance the efficacy of your program:

1. **Natural Language Processing (NLP):** 
   - **Sentiment Analysis:** Identify the tone of the messages. Positive or informative responses could be more valuable.
   - **Named Entity Recognition (NER):** Extract names, places, organizations, etc., from the text. This can help in categorizing questions and answers.
   - **Topic Modeling:** Use algorithms like Latent Dirichlet Allocation (LDA) to find topics within the text data. This can help in clustering similar questions and answers.
   - **Language Modeling:** Use models like BERT or GPT to understand the context of the conversation better.

2. **Question and Answer Matching:**
   - Develop a system to match questions with their most relevant answers.
   - Use cosine similarity or other text similarity measures to find answers that are most relevant to a given question.

3. **Ranking Algorithm:**
   - Implement a ranking algorithm that takes into account various factors like the length of the answer, the number of upvotes (if available), and the relevance to the question.
   - Consider using machine learning models to predict the usefulness of an answer based on historical data.

4. **User Feedback Integration:**
   - If possible, incorporate user feedback like upvotes, downvotes, or comments to understand the value of questions and answers.
   - Use this feedback as a part of the training data for your machine learning models.

5. **Data Preprocessing:**
   - Clean the data by removing irrelevant content, such as advertisements or off-topic messages.
   - Normalize text data by lowercasing, removing punctuation, and correcting typos.

6. **Handling Ambiguity and Sarcasm:**
   - This is a challenging aspect, but advanced NLP techniques and contextual understanding can help in identifying such nuances.

7. **Testing and Validation:**
   - Regularly test your model with new data and validate its effectiveness.
   - Consider using A/B testing to compare different models or algorithms.

8. **Scalability:**
   - Ensure your solution can handle large amounts of data without significant performance issues.
   - Optimize database queries and use efficient data structures.

9. **Ethical and Privacy Considerations:**
   - Ensure that the data usage complies with privacy laws and ethical standards.
   - Anonymize personal data where necessary.

10. **Continuous Learning and Updating:**
    - Regularly update your model with new data.
    - Consider implementing a feedback loop where the model learns from its successes and mistakes.

Remember, the effectiveness of these methods can vary based on the specific characteristics of your data and the nature of the chat group. Experimentation and iterative improvements will be key to developing a robust solution.

In [20]:
from textblob import TextBlob

def is_question(text):
    # Create a TextBlob from the input text
    blob = TextBlob(text)
    
    # Access the tags property for part-of-speech tagging
    tags = blob.tags
    
    # Check if the text contains any words tagged as WH-pronouns
    question_words = ['WDT', 'WP', 'WP$', 'WRB']

    return any(tag[1] in question_words for tag in tags)


def is_answer(text):
    # Create a TextBlob from the input text
    blob = TextBlob(text)
    
    # Access the tags
    tags = blob.tags
    
    # Check if the text contains characteristics indicative of answers
    answer_indicators = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ',  # Verb forms
                         'PRP', 'PRP$', 'DT', 'IN']  # maybe not needed lmao
                         
    return any(tag[1] in answer_indicators for tag in tags) and len(blob.sentences) > 0

# Random shit
# test_results_answers = {text: is_answer(text) for text in tg_chat_df_sample['text']}
# answers = [text for text, answer in test_results_answers.items() if answer == True]

results = []
# store the results in a dictionary
for message in range(len(tg_chat_df)):
    # print(tg_chat_df['text'][message], tg_chat_df['sender_id'][message], is_question(tg_chat_df['text'][message]))
    q = is_question(tg_chat_df['text'][message])
    a = is_answer(tg_chat_df['text'][message])
    results.append({"is_question":q, "is_answer":a,
                    "is_reply":tg_chat_df['reply'][message],
                    "message":tg_chat_df['text'][message], 
                    "sender_id":tg_chat_df['sender_id'][message],
                    "message_id":tg_chat_df['message_id'][message],
                    "reply_to_message_id":tg_chat_df['reply_to_message_id'][message],
                    "date":tg_chat_df['date'][message],
                    })

# create a dataframe from the results
results_df = pd.DataFrame(results)

In [22]:
from gensim import corpora

# Load your DataFrame just questions
df = results_df[results_df['is_question'] == True]
# Preprocess text data
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

def preprocess_text(text):
    words = word_tokenize(text)
    words = [ps.stem(word.lower()) for word in words if word.isalpha() and word.lower() not in stop_words]
    return words

df['processed_text'] = df['message'].apply(preprocess_text)


/var/folders/qm/qxx0f2_j4pd9p5_p096pjflw0000gn/T/ipykernel_18804/122245910.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['processed_text'] = df['message'].apply(preprocess_text)


In [23]:
# Create a dictionary
dictionary = corpora.Dictionary(df['processed_text'])

# Create a corpus
corpus = [dictionary.doc2bow(text) for text in df['processed_text']]


In [27]:
dictionary[0]

'behaviour'

In [25]:
from gensim.models import LdaModel
from gensim.models import Phrases

# Set the number of topics
num_topics = 15

# Build the LDA model
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=100)

# Print the topics
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)


(0, '0.026*"reward" + 0.020*"sure" + 0.015*"run" + 0.015*"look" + 0.015*"drop"')
(1, '0.027*"vote" + 0.018*"happen" + 0.014*"block" + 0.014*"need" + 0.014*"random"')
(2, '0.026*"get" + 0.021*"arm" + 0.021*"tri" + 0.017*"label" + 0.017*"scrape"')
(3, '0.023*"readjsonfromfil" + 0.018*"import" + 0.018*"build" + 0.014*"u" + 0.014*"much"')
(4, '0.032*"depend" + 0.032*"tri" + 0.018*"version" + 0.016*"goal" + 0.014*"reject"')
(5, '0.016*"pool" + 0.014*"mine" + 0.010*"look" + 0.010*"one" + 0.010*"miss"')
(6, '0.051*"see" + 0.021*"happen" + 0.017*"build" + 0.016*"thank" + 0.014*"let"')
(7, '0.036*"look" + 0.029*"btc" + 0.022*"etf" + 0.022*"like" + 0.015*"profit"')
(8, '0.023*"want" + 0.019*"like" + 0.019*"run" + 0.019*"tri" + 0.014*"make"')
(9, '0.024*"need" + 0.020*"tri" + 0.016*"well" + 0.016*"one" + 0.012*"relay"')
(10, '0.016*"compil" + 0.016*"iohk" + 0.011*"scrypt" + 0.011*"librari" + 0.011*"someon"')
(11, '0.023*"file" + 0.018*"show" + 0.014*"think" + 0.014*"want" + 0.014*"may"')
(12, '0.

It appears we have a lot of work to do. In order to get the topics if we intend to use topics. I am going to reseach some other strategies.